In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 1. بارگذاری مجموعه داده
data = sns.load_dataset("iris")  # مثال با دیتاست Iris
X = data.drop(columns="species")  # ویژگی‌ها
y = data["species"]               # متغیر هدف

In [ ]:
# تقسیم‌بندی داده‌ها به بخش‌های آموزشی و آزمون با استفاده از Hold-out و Cross-Validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 2. ایجاد مدل درخت تصمیم با معیارهای Information Gain و Gini Index
tree_gini = DecisionTreeClassifier(criterion="gini", random_state=42)
tree_entropy = DecisionTreeClassifier(criterion="entropy", random_state=42)

tree_gini.fit(X_train, y_train)
tree_entropy.fit(X_train, y_train)

In [ ]:
# 3. تنظیم پارامترهای مدل برای بهینه‌سازی (هرس کردن درخت)
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_leaf': [1, 5, 10]
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=10)
grid_search.fit(X_train, y_train)
best_tree = grid_search.best_estimator_

In [ ]:
# 4. ارزیابی مدل‌ها و محاسبه دقت و ماتریس درهم‌ریختگی
y_pred_gini = tree_gini.predict(X_test)
y_pred_entropy = tree_entropy.predict(X_test)
y_pred_best = best_tree.predict(X_test)

print("دقت مدل Gini:", accuracy_score(y_test, y_pred_gini))
print("دقت مدل Entropy:", accuracy_score(y_test, y_pred_entropy))
print("دقت مدل بهینه:", accuracy_score(y_test, y_pred_best))

# ماتریس درهم‌ریختگی
conf_matrix = confusion_matrix(y_test, y_pred_best)
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d")
plt.title("ماتریس درهم‌ریختگی مدل بهینه")
plt.show()

# گزارش دسته‌بندی
print(classification_report(y_test, y_pred_best))

In [ ]:
# 5. پیش‌بینی متغیر پاسخ برای داده‌های آزمون
cross_val_scores = cross_val_score(best_tree, X, y, cv=10)
print("دقت 10-Fold Cross-Validation:", cross_val_scores.mean())

# مقایسه نتایج Hold-Out و Cross-Validation
print("دقت Hold-Out:", accuracy_score(y_test, y_pred_best))
print("میانگین دقت Cross-Validation:", cross_val_scores.mean())

In [ ]:
# نمایش درخت تصمیم بهینه
plt.figure(figsize=(20, 10))
plot_tree(best_tree, filled=True, feature_names=X.columns, class_names=best_tree.classes_)
plt.title("درخت تصمیم بهینه")
plt.show()